In [3]:
import pandas as pd

In [4]:
# !pip install lyricsgenius
from lyricsgenius import Genius
import pandas as pd
import numpy as np

genius = Genius(
    "7Ise90s_tMVx4jQroYGWIOaskQOTBFiCh3G9J8CtgLKBK65cXk3HNFEVMYNMoGBd")


In [5]:
charts = pd.read_csv('datasets/charts.csv')
songs_and_artist = charts[['song', 'weeks-on-board', 'artist']].groupby(
    ['song', 'artist']).max().sort_values(by='weeks-on-board', ascending=False).head(50)
songs_and_artist.reset_index(level=['song', 'artist'], inplace=True)
songs_and_artist.head()

,song,artist,weeks-on-board
0,Radioactive,Imagine Dragons,87
1,Blinding Lights,The Weeknd,83
2,Sail,AWOLNATION,79
3,I'm Yours,Jason Mraz,76
4,How Do I Live,LeAnn Rimes,69


In [6]:
# https://api.discogs.com/database/search?release_title=Blinding+Lights&artist=The+Weeknd&key=inTtqFRUynYdTaZnVvYz&secret=CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR

In [7]:
import requests
import json
import sys

from urllib import request
from urllib.parse import parse_qsl
from urllib.parse import urlparse

import oauth2 as oauth

# discogs API genres info

# Your consumer key and consumer secret generated by discogs when an application is created
# and registered . See http://www.discogs.com/settings/developers .
# consumer_key = 'inTtqFRUynYdTaZnVvYz'
# consumer_secret = 'CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR'

URL = 'https://api.discogs.com/database/search?'
api_key = 'inTtqFRUynYdTaZnVvYz'
api_secret = 'CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR'
def search(song, artist):
    params = {
        'release_title' :song.replace('+',' '),
        'artist': artist,
        'key': api_key,
        'secret': api_secret
    }
    response = requests.get(URL, params=params)
    # commontrack_id = response.text
    print(response.url)
    return response.json()


In [8]:
# response.text

In [9]:
# import pylast

# # You have to have your own unique two values for API_KEY and API_SECRET
# # Obtain yours from https://www.last.fm/api/account/create for Last.fm
# API_KEY = "c946aa26f8076b3f9dbabf54f72cee96"  
# API_SECRET = "d839a53438f89d4edbf29042be4d1c26"

# # In order to perform a write operation you need to authenticate yourself
# username = "christyyywu"
# password_hash = pylast.md5("Qwer1234&")

# network = pylast.LastFMNetwork(
#     api_key=API_KEY,
#     api_secret=API_SECRET,
#     username=username,
#     password_hash=password_hash,
# )

In [10]:
# import requests
# URL = 'https://api.musixmatch.com/ws/1.1/track.search?'
# api_key = 'c446c53140147b0285b00171e61b09ca'
# def search(song, artist):
#     params = {
#         'q' : (song + ' ' + artist).replace('+',' '),
#         'apikey': api_key
#     }
#     response = requests.get(URL, params=params)
#     # commontrack_id = response.text
#     # print(response.url)
#     return response.json()


In [11]:

songs_and_artist['Query'] = songs_and_artist.apply(lambda x: search(x.song, x.artist),axis=1)

https://api.discogs.com/database/search?release_title=Radioactive&artist=Imagine+Dragons&key=inTtqFRUynYdTaZnVvYz&secret=CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR
https://api.discogs.com/database/search?release_title=Blinding+Lights&artist=The+Weeknd&key=inTtqFRUynYdTaZnVvYz&secret=CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR
https://api.discogs.com/database/search?release_title=Sail&artist=AWOLNATION&key=inTtqFRUynYdTaZnVvYz&secret=CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR
https://api.discogs.com/database/search?release_title=I%27m+Yours&artist=Jason+Mraz&key=inTtqFRUynYdTaZnVvYz&secret=CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR
https://api.discogs.com/database/search?release_title=How+Do+I+Live&artist=LeAnn+Rimes&key=inTtqFRUynYdTaZnVvYz&secret=CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR
https://api.discogs.com/database/search?release_title=Party+Rock+Anthem&artist=LMFAO+Featuring+Lauren+Bennett+%26+GoonRock&key=inTtqFRUynYdTaZnVvYz&secret=CshyGbZvbkPxHiJuxfuqNEYdAQhqGwGR
https://api.discogs.com/database/search?release_title=Counting

In [12]:
songs_and_artist_copy =  songs_and_artist.copy()

In [13]:
import re
def func(Query):
    Q = re.findall(r"'genre': \['(\w+)'\]", str(Query))
    return Q
songs_and_artist_copy['Genres'] = songs_and_artist_copy.apply(lambda row: func(row['Query']), axis=1)
# songs_and_artist

In [14]:
from collections import Counter
def counter(row):
    if len(row)>0:
        return max(set(row), key=row.count)
    else:
        return 'None'
    # return dict.C[0]
songs_and_artist_copy['Genre'] = songs_and_artist_copy.apply(lambda row: counter(row['Genres']), axis=1)
songs_and_artist_copy.head()

,song,artist,weeks-on-board,Query,Genres,Genre
0,Radioactive,Imagine Dragons,87,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...","[Rock, Rock, Rock, Rock, Rock, Rock]",Rock
1,Blinding Lights,The Weeknd,83,"{'pagination': {'page': 1, 'pages': 2, 'per_pa...","[Electronic, Electronic, Electronic, Pop, Pop]",Electronic
2,Sail,AWOLNATION,79,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...",[Electronic],Electronic
3,I'm Yours,Jason Mraz,76,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...","[Pop, Rock, Electronic, Rock, Reggae]",Rock
4,How Do I Live,LeAnn Rimes,69,"{'pagination': {'page': 1, 'pages': 1, 'per_pa...","[Electronic, Electronic, Pop, Electronic, Elec...",Electronic


In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize  
seed = 42
features = ['weeks-on-board'] 


X = songs_and_artist_copy[features] 
y = songs_and_artist_copy['Genre']

seed = 42
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


# X_train = train['Pclass', 'Sex', 'Age' ,'SibSp','Parch','Fare','Title'] 
# X_test = test['Pclass', 'Sex', 'Age' ,'SibSp','Parch','Fare','Title'] 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
forest = RandomForestClassifier(n_estimators=153, max_depth=20, random_state=1)
forest.fit(X_train, y_train)

# predict
y_pred = forest.predict(X_train)

# evaluate
print('Accuracy Score: ', forest.score(X_train, y_train))
# print('\nConfusion Matric: ', confusion_matrix(y_train, y_pred))
# print('\nClassification Report: ', classification_report(y_train, y_pred))

Accuracy Score:  0.65


In [18]:
from sklearn.metrics import plot_confusion_matrix , classification_report
print(classification_report(y_test,y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [10, 40]

In [166]:
# import json

# result = json.loads(songs_and_artist.Query[2])

# id, genre = list(), list()
# for track in result['message']['body']['track_list']:
#     try:
#         print(track['track']['commontrack_id'])
#         print(track['track']['primary_genres']['music_genre_list'][0]['music_genre']['music_genre_name'])
#         id.append(track['track']['commontrack_id'])
#         genre.append(track['track']['primary_genres']
#                     ['music_genre_list'][0]['music_genre']['music_genre_name'])
#     except:
#         genre.append(None)
#         id.append(None)

# print(id, genre)

In [ ]:
# import json
# result = json.loads(songs_and_artist.Query[2])
# json.load()songs_and_artist.Query[0]

In [106]:
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.pipeline import Pipeline
# from sklearn.svm import LinearSVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from nltk.tokenize import word_tokenize  
# seed = 42

In [19]:
# drop songs of which lyrics can't be found  
songs_and_artist_1 = songs_and_artist[songs_and_artist['lyrics'] != 'empty'] 
songs_and_artist_1

KeyError: 'lyrics'

In [124]:

# Define X and y
X = songs_and_artist_1['lyrics'].to_list()
y = songs_and_artist_1['weeks-on-board'].to_list()


# split train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed)


# Initialize pipeline
pipe = Pipeline([('count_vec', CountVectorizer(stop_words="english", analyzer='word',
                                               ngram_range=(1, 1), max_df=1.0,
                                               min_df=1, max_features=None)),
                 ('svm', LinearSVC())])


pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)


In [123]:
from sklearn.metrics import plot_confusion_matrix , classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          53       0.00      0.00      0.00       1.0
          54       0.00      0.00      0.00       2.0
          55       0.00      0.00      0.00       0.0
          56       0.00      0.00      0.00       2.0
          57       0.00      0.00      0.00       2.0
          58       0.00      0.00      0.00       1.0
          59       0.00      0.00      0.00       0.0
          61       0.00      0.00      0.00       1.0
          62       0.00      0.00      0.00       0.0
          68       0.00      0.00      0.00       1.0
          69       0.00      0.00      0.00       0.0
          83       0.00      0.00      0.00       0.0

    accuracy                           0.00      10.0
   macro avg       0.00      0.00      0.00      10.0
weighted avg       0.00      0.00      0.00      10.0



/Users/yc/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yc/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yc/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yc/opt/anaconda3/lib/python3.

In [129]:
len(y_test)

10